In [8]:
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import TypedDict
from dotenv import load_dotenv

In [9]:
load_dotenv()

True

In [10]:
model=ChatGoogleGenerativeAI(model='gemini-1.5-flash')

In [11]:
class BlogState(TypedDict):
    title: str
    outline: str
    content: str
    score: str

In [ ]:
def create_outline(state: BlogState)-> BlogState:
    title=state['title']
    prompt=f"Generate a detailed outline for a blog on the topic {title}"
    outline=model.invoke(prompt).content
    state['outline']=outline
    return state

def create_blog(state: BlogState)-> BlogState:
    title=state['title']
    outline=state['outline']
    prompt=f"Write a detailed blog on the title - {title} using the following outline - {outline}"
    content=model.invoke(prompt).content

    state['content']=content
    return state

def generate_score(state: BlogState)-> BlogState:
    outline=state['outline']
    content=state['content']
    prompt=f"""
    Generate a match score ranging from 0 to 100 based on the outline and content describing the summarization match between content from outline - content:{content}, outline: {outline}.
    Give me the matching score only, no description needed. And give me that score round off in two decimal places
    """
    score=model.invoke(prompt).content
    state['score']=score
    return state


In [13]:
graph=StateGraph(BlogState)
graph.add_node('create_outline', create_outline)
graph.add_node('create_blog', create_blog)
graph.add_node('generate_score', generate_score)

graph.add_edge(START, 'create_outline')
graph.add_edge('create_outline','create_blog')
graph.add_edge('create_blog', 'generate_score')
graph.add_edge('generate_score', END)

workflow=graph.compile()


In [15]:
initial_state={'title': 'Omuamua and 3I Atlas'}

final_state=workflow.invoke(initial_state)
print(final_state)

{'title': 'Omuamua and 3I Atlas', 'outline': '## Blog Outline: Oumuamua & 3I/Atlas: Interstellar Visitors & Their Mysteries\n\n**I. Introduction (approx. 100 words)**\n\n* Hook: Start with a captivating sentence about the possibility of interstellar objects visiting our solar system.\n* Briefly introduce Oumuamua and 3I/Atlas as the first two confirmed interstellar objects.\n* Highlight the significance of their discovery for our understanding of interstellar space and planetary formation.\n* Thesis statement: This blog will explore the unique characteristics of Oumuamua and 3I/Atlas, compare and contrast their properties, and discuss the ongoing scientific debate surrounding their origins and nature.\n\n\n**II. Oumuamua: The Mysterious Interstellar Wanderer (approx. 300 words)**\n\n* **A. Discovery and Initial Observations:**\n    * Date of discovery and who discovered it.\n    * Initial classification as an asteroid, then its unusual trajectory.\n    *  Mention its highly elongated s